# Test download handler

In [1]:
%run ./setup-logging.ipynb
%run ./jupyter-test-support.ipynb
%run ./create_container.ipynb

Created [container]
  * app_root
  * blob_root_directory
  * blob_store
  * download_request_store
  * environment_name
  * event_publisher
  * mongodb_client
  * mongodb_credentials
  * mongodb_db_name
  * queue_receiver
  * queue_receiver_manager
  * queue_sender
  * rabbitmq_connection
  * rabbitmq_url
  * robot_store
  * url_download_handler
  * url_downloaded_handler
  * url_info_store
  * url_store


In [2]:
%%cell_testsuite

@cell_test
def test_download(self):
    download_handler = container.url_download_handler()
    url_store = container.url_store()
    url = 'https://www.bing.com/robots.txt'
    download_handler({ 'url': url, 'fire_downloaded': False })
    url_info, content = url_store.get(url, lambda url_info, fh: (url_info, fh.read()))
    self.assertTrue(len(content) > 0)
    print(len(content))

DEBUG|2020-11-25 21:31:42,459|MainThread|rook.wcrawl.handlers.url_download_handler|Request to download: {'url': 'https://www.bing.com/robots.txt', 'fire_downloaded': False}
2456
.
----------------------------------------------------------------------
Ran 1 test in 0.270s

OK


In [3]:
robot_store = container.robot_store()
download_handler = container.url_download_handler()

In [4]:
%%cell_testsuite

@cell_test
def test_domain_with_robots(self):
    url = 'https://towardsdatascience.com/part-of-speech-tagging-for-beginners-3a0754b2ebba?source=collection_home---------5----------------------------'
    rf = robot_store.get_robots(url, download_handler)
    
    self.assertTrue(rf.can_fetch('*', url))
    
    disallowed_paths = [rl.path for rl in rf.default_entry.rulelines if not rl.allowance]
    self.assertFalse(rf.can_fetch('*', 'https://towardsdatascience.com' + disallowed_paths[0]))

{'content_type': 'text/plain; charset=utf-8', 'retrieved': datetime.datetime(2020, 11, 26, 2, 34, 13, 109000), 'url': 'https://towardsdatascience.com/robots.txt'}
.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK
